In [1]:
import pandas as pd
from dotenv import load_dotenv
import requests
import os
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver

# Tide-up the original dataframe

In [3]:
fifa_df_2 = pd.read_csv('csv/players_19.csv')
fifa_df_2 = fifa_df_2[fifa_df_2['overall'] >= 85]

In [4]:
fifa_df_2.head(3)

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,33,1985-02-05,187,83,Portugal,Juventus,...,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3
1,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,31,1987-06-24,170,72,Argentina,FC Barcelona,...,63+2,61+2,61+2,61+2,63+2,58+2,47+2,47+2,47+2,58+2
2,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Junior,26,1992-02-05,175,68,Brazil,Paris Saint-Germain,...,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3


In [5]:
fifa_df_2.info(max_cols=105)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111 entries, 0 to 110
Data columns (total 104 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   sofifa_id                   111 non-null    int64  
 1   player_url                  111 non-null    object 
 2   short_name                  111 non-null    object 
 3   long_name                   111 non-null    object 
 4   age                         111 non-null    int64  
 5   dob                         111 non-null    object 
 6   height_cm                   111 non-null    int64  
 7   weight_kg                   111 non-null    int64  
 8   nationality                 111 non-null    object 
 9   club                        111 non-null    object 
 10  overall                     111 non-null    int64  
 11  potential                   111 non-null    int64  
 12  value_eur                   111 non-null    int64  
 13  wage_eur                    111 no

In [6]:
fifa_19_pdata = fifa_df_2.iloc[:,[0,2,3,4,8,9,24]]
fifa_19_pstats = fifa_df_2.iloc[:, np.r_[0,10,11, 31:43]]

In [7]:
fifa_19_pdata

,sofifa_id,short_name,long_name,age,nationality,club,team_position
0,20801,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,33,Portugal,Juventus,LW
1,158023,L. Messi,Lionel Andrés Messi Cuccittini,31,Argentina,FC Barcelona,RW
2,190871,Neymar Jr,Neymar da Silva Santos Junior,26,Brazil,Paris Saint-Germain,CAM
3,193080,De Gea,David De Gea Quintana,27,Spain,Manchester United,GK
4,192985,K. De Bruyne,Kevin De Bruyne,27,Belgium,Manchester City,RCM
...,...,...,...,...,...,...,...
106,184267,Y. Brahimi,Yacine Brahimi,28,Algeria,FC Porto,LM
107,187961,Paulinho,José Paulo Bezzera Maciel Júnior,29,Brazil,Guangzhou Evergrande Taobao FC,LDM
108,188350,M. Reus,Marco Reus,29,Germany,Borussia Dortmund,LW
109,192629,Iago Aspas,Iago Aspas Juncal,30,Spain,RC Celta,RW


In [8]:
fifa_19_pstats.head()

,sofifa_id,overall,potential,pace,shooting,passing,dribbling,defending,physic,gk_diving,gk_handling,gk_kicking,gk_reflexes,gk_speed,gk_positioning
0,20801,94,94,90.0,93.0,81.0,89.0,35.0,79.0,NaN,NaN,NaN,NaN,NaN,NaN
1,158023,94,94,88.0,91.0,88.0,96.0,32.0,61.0,NaN,NaN,NaN,NaN,NaN,NaN
2,190871,92,93,92.0,84.0,83.0,95.0,32.0,59.0,NaN,NaN,NaN,NaN,NaN,NaN
3,193080,91,93,NaN,NaN,NaN,NaN,NaN,NaN,90.0,85.0,87.0,94.0,58.0,88.0
4,192985,91,92,77.0,86.0,92.0,87.0,60.0,78.0,NaN,NaN,NaN,NaN,NaN,NaN


# Get players links from fbref.com

In [20]:
player_link_df = fifa_19_pdata.iloc[:,[0,1]]
player_link_df['short_name'] = player_link_df['short_name'].str.lower().str.replace('.', '').str.replace(' ', '+')
player_link_df

,sofifa_id,short_name
0,20801,cristiano+ronaldo
1,158023,l+messi
2,190871,neymar+jr
3,193080,de+gea
4,192985,k+de+bruyne
...,...,...
106,184267,y+brahimi
107,187961,paulinho
108,188350,m+reus
109,192629,iago+aspas


In [21]:
player_link_df_try = player_link_df.iloc[:10,:]

In [22]:
op = webdriver.ChromeOptions()
op.add_argument('headless')

In [26]:
browser = webdriver.Chrome('/Users/eduardooportoalonso/Documents/Cursos/Ironhack/datamad1020/ironhack-projects/pipelines-project/chromedriver', options=op)

def linkifyer(search_str, browser):
    browser.get("https://fbref.com/en/search/search.fcgi?search=" + search_str) 

    element = browser.find_element_by_class_name('filter')
    selector = browser.execute_script("return arguments[0].innerHTML;", element)
    link = BeautifulSoup(selector, 'html.parser').find('a')['href']
    print(link)

In [27]:
for name in player_link_df_try['short_name']:
    linkifyer(name, browser)

/en/players/dea698d9/all_comps/Cristiano-Ronaldo-Stats---All-Competitions
javascript:void(0)
/en/players/69384e5d/all_comps/Neymar-Stats---All-Competitions
javascript:void(0)
/en/players/e46012d4/all_comps/Kevin-De-Bruyne-Stats---All-Competitions
javascript:void(0)
javascript:void(0)
/en/players/6025fab1/all_comps/Luka-Modric-Stats---All-Competitions
/en/players/a39bb753/all_comps/Eden-Hazard-Stats---All-Competitions
/en/players/ee8508c0/all_comps/Jan-Oblak-Stats---All-Competitions
